# 自动站数据转换

In [14]:
import pandas as pd
import numpy as np
import xarray as xr
station_Mws = pd.read_csv("Mws2022.csv", keep_default_na=-9999)
station_Aws = pd.read_csv("Aws2022.csv", keep_default_na=-9999)
station_all = pd.concat([station_Aws,station_Mws])

grouped = station_all.groupby('IIiii')

# 根据time lat lon RR 对数据进行转换



# Class版的数据处理

In [7]:
import pandas as pd
import numpy as np
from math import isnan
sql = 'test'  

class sql_data:
    def __init__(self, sql):
        self.sql = sql# 传进来的参数
        station_Mws = pd.read_csv("Mws2022.csv")
        station_Aws = pd.read_csv("Aws2022.csv")
        self.station_all = pd.concat([station_Aws,station_Mws])
        # 数据
        self.grouped_county = self.station_all.groupby('county')
        self.grouped_IIiii = self.station_all.groupby('IIiii')
        self.timecounts = len(self.grouped_IIiii.get_group(58660)['tTime'])
        # 测试数据
        print(self.timecounts)
    def comput_county(self):
        '计算面最大雨强、累计降水、最高、最低气温'
        self.station_county_comput = []
        for i in self.grouped_county.size().index:  
            data= self.grouped_county.get_group(i)
            data['VV'].replace(-9999,np.nan,inplace=True)
            data['RR'].replace(-9999,np.nan,inplace=True)
            data['Tn'].replace(-9999,np.nan,inplace=True)
            data['Tx'].replace(-9999,np.nan,inplace=True)
            dic = {}
            dic['county'] = str(i)
            dic['RR'] = data['RR'].mean()*self.timecounts/10.0
            dic['RMax'] = data['RR'].max()/10.0
            dic['Tx'] = data['Tx'].max()/10.0
            dic['Tn'] = data['Tn'].min()/10.0 
#             print(dic)
            self.station_county_comput.append(dic)
        tmp_max_County = []
        tmp_min_County = []
        RR_County = []
        for i in self.station_county_comput:
            tmp_max_County.append({"name":i['county'],"value":i['Tx']})
            tmp_min_County.append({"name":i['county'],"value":i['Tn']})
            RR_County.append({"name":i['county'],"value":i['RR']})
        return RR_County,tmp_max_County,tmp_min_County

    def comput_IIiii(self):
        self.station_dot_comput = {} 
        lat = []
        lon = []
        tx = []
        tn = []
        rr = [] 
        rx = []
        county = []
        name = []
        vv = []
        town = []
        vv_min = " "#{"name":[],"value":0.0}
        vv_value = 1000.0
        station_RR_blue = 0.0
        station_RR_yellow = 0.0
        station_RR_black = 0.0
        station_RR_red = 0.0
        station_RR_green = 0.0
        station_vv = []
        for i in self.grouped_IIiii.size().index:
            data= self.grouped_IIiii.get_group(i)
            #print(data)
            data['VV'].replace(-9999,np.nan,inplace=True)
            data['RR'].replace(-9999,np.nan,inplace=True)
            data['Tn'].replace(-9999,np.nan,inplace=True)
            data['Tx'].replace(-9999,np.nan,inplace=True)
            dic = {}
            dic['IIiii'] = data['IIiii'].iloc[0]
            dic['tTime'] = data['tTime'].tolist()
            dic['StationName'] = data['StationName'].iloc[0]
            dic['county'] = data['county'].iloc[0]
            dic['lat'] = data['lat'].iloc[0]
            dic['lon'] = data['lon'].iloc[0]
            dic['Town'] = data['Town'].iloc[0]
            dic['fFy'] = data['fFy'].max()           
            dic['dFy'] = data[data['fFy']== data['fFy'].max()]['dFy'].iloc[0]
            dic['T'] = data['T'].tolist()
            dic['VList'] = data['VV'].tolist()
            dic['Tx'] = data['Tx'].max()/10.0
                        
            dic['Tn'] = data['Tn'].min()/10.0
            if not isnan(data['VV'].min()):
                #统计能见度自动站名称
                station_vv.append(data['IIiii'].iloc[0])
                if data['VV'].min() < vv_value:
                    print(data['IIiii'].iloc[0])
                    vv_min=  str(data['IIiii'].iloc[0])
                    vv_value = data['VV'].min()
            dic['VV'] = data['VV'].min()
            dic['RR'] = data['RR'].sum()
            dic['RMax'] = data['RR'].max()
            # 降水分级别
            if not isnan(data['RR'].sum()):
                value_rsum = data['RR'].sum()
                if value_rsum >=0 and value_rsum <10:
                    station_RR_blue =  station_RR_blue +1   
                elif value_rsum >=0 and value_rsum <50:
                    station_RR_yellow =  station_RR_yellow+1   
                elif value_rsum >=50 and value_rsum <100:
                    station_RR_black =  station_RR_black+1   
                elif value_rsum >=50 and value_rsum <100:
                    station_RR_red =  station_RR_red+1   
                else:
                    station_RR_green =  station_RR_green +1  
                    
                
            
            lat.append(data['lat'].iloc[0])
            town.append(data['Town'].iloc[0])
            lon.append(data['lon'].iloc[0])
            county.append(data['county'].iloc[0])
            name.append(data['IIiii'].iloc[0])
            tx.append(data['Tx'].max()/10.0)
            tn.append(data['Tn'].max()/10.0)
            rr.append(data['RR'].sum()/10.0)
            rx.append(data['RR'].max()/10.0)
#             print(dic)
            self.station_dot_comput[str(i)] = dic 
#         print(lat)
        #排序数据
        data_rx =  pd.DataFrame()
        data_rx['name'] = name 
        data_rx['county'] = county
        data_rx['town'] = town
        data_rx['rx'] = rx 
#         print(data_rx.sort_values(by =['rx'],ascending = [False]))
        data_rsum =  pd.DataFrame()
        data_rsum['name'] = name 
        data_rsum['county'] = county
        data_rsum['town'] = town
        data_rsum['rsum'] = rr 
#         print(data_rx.sort_values(by =['rx'],ascending = [False]))
        # 最大值对应的站点序列
        data_vv = vv_min
        #print("最低能见度",data_vv,vv_min)
        data_vvmin =  pd.DataFrame()
        data_vvmin['tTime']= self.station_dot_comput[data_vv]['tTime']
        data_vvmin['VV']= self.station_dot_comput[data_vv]['VList']
        #print("data:",data_vvmin)
        # 计算分级
        print("data:",station_RR_green,station_RR_red,station_RR_yellow,station_RR_black)
        # 计算scatter
        return lat,lon,tx,tn,rr,rx,vv,data_rx.sort_values(by =['rx'],ascending = [False]),data_rsum,data_vvmin
    

    

    
sql_data(sql).comput_IIiii()
b = pd.DataFrame()
b['lat'] = [1,2]
print(b)
# a =  sql_data(sql).station_all
%time

733
58559
58663
58668
K8719
data: 450.0 0.0 0.0 0.0
   lat
0    1
1    2
CPU times: user 16 µs, sys: 1 µs, total: 17 µs
Wall time: 153 µs
